## 패키지 import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

## 모델 적합 - LinearRegression, RF, XGB, LGBM

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error, r2_score

merged_df = pd.read_csv('data/train_data2.csv').drop('Unnamed: 0', axis=1)

X, y = merged_df.drop(['stock_id','time_id','target'], axis=1), merged_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

### Random Forest

In [ ]:
##### rf = RandomForestRegressor()

n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 5)]
max_features = ['auto','sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
min_samples_split = [5, 10]
min_samples_leaf = [2, 4]
bootstrap = [True]

rf_param = {'n_estimators' : n_estimators,
            'max_features' : max_features,
            'max_depth' : max_depth,
            'min_samples_split' : min_samples_split,
            'min_samples_leaf' : min_samples_leaf,
            'bootstrap' : bootstrap}

random_rf = RandomizedSearchCV(rf, param_distributions = rf_param,n_iter = 100, verbose = 2, scoring = 'neg_mean_squared_error', cv = 4, n_jobs=-1)
random_rf.fit(X_train, y_train)

In [5]:
print(random_rf.best_params_)
print(np.round(random_rf.best_score_, 4))
best_rf = random_rf.best_estimator_

{'n_estimators': 65, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': True}
-0.0


### -0.0은 뭔소리인지.

In [5]:
# n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 5)]
# max_features = ['auto','sqrt']
# max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
# min_samples_split = [5, 10]
# min_samples_leaf = [2, 4]
# bootstrap = [True]

# rf_param = {'n_estimators' : n_estimators,
#             'max_features' : max_features,
#             'max_depth' : max_depth,
#             'min_samples_split' : min_samples_split,
#             'min_samples_leaf' : min_samples_leaf,
#             'bootstrap' : bootstrap}
#################################################
###### 위에는 신경쓰지마시오 #######################
#################################################

best_rf = RandomForestRegressor(n_estimators=65, max_depth=10, min_samples_split=5,
                                min_samples_leaf=4, max_features='auto', bootstrap=True)
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)
print('RMSE :', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2 :', r2_score(y_test, y_pred))
y_test_df = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})
y_test_df['rmspe_before'] = ((y_test_df.y_test-y_test_df.y_pred)/y_test_df.y_test)**2
print('RMSPE :',np.sqrt(y_test_df.rmspe_before.sum()/len(y_test_df)))

RMSE : 0.0013412178918026545
R2 : 0.7930008799753355
RMSPE : 0.2915957574822583


In [11]:
pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})

,y_test,y_pred
133441,0.009850,0.005394
1656,0.002723,0.001856
201901,0.002381,0.003168
251007,0.002371,0.001959
315101,0.002075,0.002034
413282,0.003575,0.003307
161859,0.000888,0.001696
151471,0.003018,0.002399
327673,0.009047,0.005462
418657,0.004924,0.005418


In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
max_features = ['auto','sqrt']
max_depth = [10]
min_samples_split = [2, 5]
min_samples_leaf = [4]
bootstrap = [True]

rf_param = {'n_estimators' : n_estimators,
            'max_features' : max_features,
            'max_depth' : max_depth,
            'min_samples_split' : min_samples_split,
            'min_samples_leaf' : min_samples_leaf,
            'bootstrap' : bootstrap}

random_rf2 = RandomizedSearchCV(rf, param_distributions = rf_param,n_iter = 100, verbose = 2, scoring = 'neg_mean_squared_error', cv = 4, n_jobs=-1)
random_rf2.fit(X_train, y_train)

C:\Users\frank\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:289: UserWarning: The total space of parameters 20 is smaller than n_iter=100. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Fitting 4 folds for each of 20 candidates, totalling 80 fits


In [ ]:
print(random_rf2.best_params_)
print(np.round(random_rf2.best_score_, 4))
best_rf2 = random_rf2.best_estimator_

In [ ]:
from xgboost import XGBRegressor

model = XGBClassifier(파라미터들)
xgb_model = model.fit(x_train, y_train, early_stopping_rounds=100, 
                        eval_metric='logloss',eval_set=[(X_val, y_val)])

# 예측하기
y_pre = xgb_model.predict(X_test)
y_pred_probs = xgb_model.predict_proba(X_test)[:,1]

# 특성 중요도 시각화
fig, ax = plt.subplots(figsize=(10,12))
plot_importance(xgb_model, ax=ax)

## XGBoost 적합

In [26]:
eta = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]
num_boost_rounds = np.linspace(100, 1000, num=5)
min_child_weight = 1
gamma = 0
max_depth = 6
sub_sample = 0.8


array([ 100.,  325.,  550.,  775., 1000.])

In [8]:
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [6]:
test.head()

,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [7]:
sample_submission.head()

,row_id,target
0,0-4,0.003048
1,0-32,0.003048
2,0-34,0.003048


In [15]:
submission_df = pd.DataFrame({'row_id' : merged_df['target'], 'target':y_pred})

ValueError: array length 85787 does not match index length 428932

In [19]:
merged_df[(merged_df.stock_id == 61) & (merged_df.time_id >1700) & (merged_df.time_id < 5000)]

,stock_id,time_id,total_ratio,real_vol,skewness,right_skew,book_row_num,bid_size1,ask_size1,bid_size2,ask_size2,traded_size,trade_TR,max_size,max_size_price,max_price,max_price_size,target
207004,61.0,1707.0,0.829457,0.005058,-0.616585,148.900735,516.0,100.0,100.0,1.0,85.0,24675.0,0.006254,3210.0,0.996143,1.001925,8.0,0.005734
207005,61.0,1712.0,0.732997,0.001969,-0.890175,146.686047,397.0,1.0,13.0,3.0,3.0,4212.0,0.001239,349.0,1.000339,1.000758,145.0,0.001369
207006,61.0,1719.0,0.709746,0.004460,0.442266,139.462882,472.0,2.0,1.0,1.0,50.0,19465.0,0.003513,2751.0,0.997460,1.000087,1.0,0.003277
207007,61.0,1722.0,0.781193,0.009360,0.705810,149.073801,553.0,100.0,300.0,10.0,100.0,26602.0,0.009753,2509.0,0.995880,1.004188,92.0,0.009046
207008,61.0,1728.0,0.857430,0.002781,-0.419146,152.988095,498.0,100.0,100.0,100.0,269.0,13246.0,0.005250,597.0,0.999466,1.000367,14.0,0.003266
207009,61.0,1744.0,0.693833,0.004022,0.540661,147.442308,454.0,1.0,3.0,2.0,200.0,6027.0,0.001561,1269.0,1.000836,1.001144,406.0,0.003274
207010,61.0,1747.0,0.787500,0.003310,0.594026,146.388430,480.0,62.0,3.0,203.0,5.0,4544.0,0.002286,479.0,1.001900,1.002555,17.0,0.003366
207011,61.0,1761.0,0.917254,0.003305,-0.478057,147.411348,568.0,11.0,2.0,17.0,1.0,43696.0,0.007604,3772.0,0.992100,0.997629,200.0,0.003730
207012,61.0,1787.0,0.868526,0.003700,0.670461,148.074219,502.0,1.0,75.0,101.0,3.0,7497.0,0.003811,702.0,0.998394,1.000687,152.0,0.003610
207013,61.0,1789.0,0.867470,0.004095,1.069982,150.239216,498.0,84.0,75.0,39.0,1.0,13595.0,0.005698,589.0,0.997808,1.001704,12.0,0.003525


In [17]:
train

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747
5,0,72,0.004912
6,0,97,0.009388
7,0,103,0.004120
8,0,109,0.002182
9,0,123,0.002669
